<a href="https://colab.research.google.com/github/skandrigi/non-stationary/blob/main/tst.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install mani_skill
!pip install stable-baselines3

In [14]:
from math import e
import gymnasium as gym
import numpy as np
from stable_baselines3 import TD3
from stable_baselines3.common.env_util import make_vec_env
from sympy import det
from mani_skill.utils import gym_utils
from mani_skill.utils.wrappers.record import RecordEpisode
from stable_baselines3.common.noise import VectorizedActionNoise, NormalActionNoise
from mani_skill.vector.wrappers.sb3 import ManiSkillSB3VectorEnv
import torch

ms3_vec_env = gym.make("PushCube-v1", num_envs=1024)
max_episode_steps = gym_utils.find_max_episode_steps_value(ms3_vec_env)
vec_env = ManiSkillSB3VectorEnv(ms3_vec_env)
n_actions = vec_env.action_space.shape[-1]
base_noise = NormalActionNoise(
    mean=np.zeros(n_actions), sigma=0.1 * np.ones(n_actions)
)
base_noise.reset()
action_noise = VectorizedActionNoise(base_noise, 1024)
model = TD3("MlpPolicy", vec_env, action_noise=action_noise, verbose=1,device='auto')
model.learn(total_timesteps=1_000_000)
model.save("ppo_pushcube")
vec_env.close()
del model

model = PPO.load("ppo_pushcube")

eval_vec_env = gym.make("PushCube-v1", num_envs=16, render_mode="rgb_array")
eval_vec_env = RecordEpisode(eval_vec_env, output_dir="eval_videos_1", save_video=True, trajectory_name="eval_trajectory", max_steps_per_video=max_episode_steps)
eval_vec_env = ManiSkillSB3VectorEnv(eval_vec_env)
obs = eval_vec_env.reset()
for i in range(max_episode_steps):
  action, _states = model.predict(obs, deterministic=True)
  obs, rewards, dones, info = eval_vec_env.step(action)

/usr/local/lib/python3.11/dist-packages/gymnasium/core.py:311: UserWarning: WARN: env.num_envs to get variables from other wrappers is deprecated and will be removed in v1.0, to get this variable you can do `env.unwrapped.num_envs` for environment variables or `env.get_wrapper_attr('num_envs')` that will search the reminding wrappers.
  logger.warn(
/usr/local/lib/python3.11/dist-packages/gymnasium/core.py:311: UserWarning: WARN: env.single_observation_space to get variables from other wrappers is deprecated and will be removed in v1.0, to get this variable you can do `env.unwrapped.single_observation_space` for environment variables or `env.get_wrapper_attr('single_observation_space')` that will search the reminding wrappers.
  logger.warn(
/usr/local/lib/python3.11/dist-packages/gymnasium/core.py:311: UserWarning: WARN: env.single_action_space to get variables from other wrappers is deprecated and will be removed in v1.0, to get this variable you can do `env.unwrapped.single_action_s

Using cuda device


TypeError: can't convert cuda:0 device type tensor to numpy. Use Tensor.cpu() to copy the tensor to host memory first.

In [ ]:
from IPython.display import Video

Video("eval_videos_1/0.mp4", embed=True, width=1024)

In [ ]:
import gymnasium as gym
from stable_baselines3 import PPO
from mani_skill.vector.wrappers.sb3 import ManiSkillSB3VectorEnv

model = PPO.load("ppo_pushcube")
new_env_id = "PullCube-v1"
num_envs = 1024
new_env = gym.make(new_env_id, num_envs=num_envs)
vec_env = ManiSkillSB3VectorEnv(new_env)
model.set_env(vec_env)
model.learn(total_timesteps=1_000_000)
model.save("ppo_pickcube")
vec_env.close()
del model


model = PPO.load("ppo_pickcube")

eval_vec_env = gym.make("PullCube-v1", num_envs=16, render_mode="rgb_array")
eval_vec_env = RecordEpisode(eval_vec_env, output_dir="eval_videos_2", save_video=True, trajectory_name="eval_trajectory", max_steps_per_video=max_episode_steps)
eval_vec_env = ManiSkillSB3VectorEnv(eval_vec_env)
obs = eval_vec_env.reset()
for i in range(max_episode_steps):
  action, _states = model.predict(obs, deterministic=True)
  obs, rewards, dones, info = eval_vec_env.step(action)

/usr/local/lib/python3.11/dist-packages/stable_baselines3/common/on_policy_algorithm.py:150: UserWarning: You are trying to run PPO on the GPU, but it is primarily intended to run on the CPU when not using a CNN policy (you are using ActorCriticPolicy which should be a MlpPolicy). See https://github.com/DLR-RM/stable-baselines3/issues/1245 for more info. You can pass `device='cpu'` or `export CUDA_VISIBLE_DEVICES=` to force using the CPU.Note: The model will train, but the GPU utilization will be poor and the training might take longer than on CPU.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/mani_skill/utils/building/actor_builder.py:215: DeprecationWarning: The 'warn' method is deprecated, use 'warning' instead
  logger.warn(
2025-04-03 01:30:31,251 - mani_skill  - WARNING - No initial pose set for actor builder of goal_region, setting to default pose q=[1,0,0,0], p=[0,0,0]. Not setting reasonable initial poses may slow down simulation, see https://github.com/haosulab/Man

---------------------------------
| rollout/           |          |
|    ep_len_mean     | 50       |
|    ep_rew_mean     | 7.49     |
|    success_rate    | 0        |
| time/              |          |
|    fps             | 7417     |
|    iterations      | 1        |
|    time_elapsed    | 6        |
|    total_timesteps | 51200    |
---------------------------------
----------------------------------------
| rollout/                |            |
|    ep_len_mean          | 50         |
|    ep_rew_mean          | 7.84       |
|    success_rate         | 0          |
| time/                   |            |
|    fps                  | 3295       |
|    iterations           | 2          |
|    time_elapsed         | 31         |
|    total_timesteps      | 102400     |
| train/                  |            |
|    approx_kl            | 0.04956998 |
|    clip_fraction        | 0.461      |
|    clip_range           | 0.2        |
|    entropy_loss         | -8.58      |
|    explai

2025-04-03 01:38:42,192 - mani_skill  - WARNING - No initial pose set for actor builder of goal_region, setting to default pose q=[1,0,0,0], p=[0,0,0]. Not setting reasonable initial poses may slow down simulation, see https://github.com/haosulab/ManiSkill/issues/421.
2025-04-03 01:38:42,247 - mani_skill  - WARNING - mani_skill is not installed with git.


In [ ]:
from IPython.display import Video

Video("eval_videos_2/0.mp4", embed=True, width=1024)